In [1]:
import logging
import sys
import warnings
from urllib.parse import urlparse

import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import os

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [2]:
# this cell is tagged "parameters"
alpha = 0.7
l1_ratio = 0.7
csv_train_path = "train.csv"
csv_test_path = "test.csv"

In [3]:
# Parameters
alpha = 0.5
l1_ratio = 0.1
csv_train_path = "mlflow-artifacts:/238050313658468910/ed82a02e83534cf6b31085c027af07e1/artifacts/train.csv"
csv_test_path = "mlflow-artifacts:/238050313658468910/ed82a02e83534cf6b31085c027af07e1/artifacts/test.csv"


In [4]:
with mlflow.start_run(tags={"mlflow.runName": "Train"}) as mlrun:
    local_train_path = mlflow.artifacts.download_artifacts(
        artifact_uri=csv_train_path, dst_path="./downloads"
    )
    local_test_path = mlflow.artifacts.download_artifacts(
        artifact_uri=csv_test_path, dst_path="./downloads"
    )
    
    train = pd.read_csv(local_train_path)
    test = pd.read_csv(local_test_path)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]   
    
    
    
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    
    predicted_qualities = lr.predict(test_x)
    
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
    
    print(f"Elasticnet model (alpha={alpha:f}, l1_ratio={l1_ratio:f}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")
    
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    
    predictions = lr.predict(train_x)
    signature = infer_signature(train_x, predictions)
    
    mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel", signature=signature)

Elasticnet model (alpha=0.500000, l1_ratio=0.100000):
  RMSE: 0.7195761947358503
  MAE: 0.565247170442462
  R2: 0.23003231594789408


Registered model 'ElasticnetWineModel' already exists. Creating a new version of this model...


2023/10/26 11:06:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetWineModel, version 3


Created version '3' of model 'ElasticnetWineModel'.
